In [1]:
import gdal
from scipy.optimize import curve_fit

In [2]:
depths = ['0_5','5_15','15_30','30_60','60_100','100_200']
mids = [2.5,10,15+7.5,45,80,150]
thicknesses = [5,10,15,30,40,100]
variables = ['silt','sand','clay','lambda','hb','ksat']
products = ['column','column','column','column','column','surface']
stat = 'mean'

In [3]:
def generate_df(variable,depths):
    
    fl = '/RHESSys/Como/auxdata/POLARIS_3arcsec/%s_%s_%s_crop_project.tif'%(variable,stat,depths[0])
    ds = gdal.Open(fl)
    dat = np.array(ds.GetRasterBand(1).ReadAsArray(),dtype='float')
    m,n = dat.shape
    #print m*n
    tmp = pd.DataFrame(index=np.arange(0,(m*n)),columns=depths) # generate data frame
    for depth in depths:
        fl = '/RHESSys/Como/auxdata/POLARIS_3arcsec/%s_%s_%s_crop_project.tif'%(variable,stat,depth)
        ds = gdal.Open(fl)
        dat = np.array(ds.GetRasterBand(1).ReadAsArray(),dtype='float')
        m,n = dat.shape
        #print m*n
        
        dat = np.reshape(dat,(m*n,1)) # reshape to a column
        
        tmp[depth] = dat
    
    return tmp

In [4]:
ksat = generate_df('ksat',depths)
clay = generate_df('clay',depths)
silt = generate_df('silt',depths)
sand = generate_df('sand',depths)
po = generate_df('lambda',depths)
pa = generate_df('hb',depths)

In [5]:
def compute_cell_averages(df):
    total_depth = np.sum(thicknesses)
    weights = np.array(thicknesses)/float(total_depth)
    
    tmp = []
    
    for weight,depth in zip(weights,depths):
        tmp.append(df[depth]*weight)
    
    return np.sum(tmp)

In [6]:
df = pd.DataFrame()

In [7]:
def estimate_m(df):
    surf = df['0_5'] # grab the surface ksat value
    
    ksats = []
    
    for name in depths: ksats.append(df[name]) # extract ksat values
        
    ksats = np.array(ksats)
    
    def exp_decay(z,m):
        return surf*np.exp((z/m)*-1)
    
    popt, pcov = curve_fit(exp_decay,ksats,mids)
    
    return popt[0]

In [8]:
df['clay'] = clay.apply(compute_cell_averages,axis=1)
df['silt'] = silt.apply(compute_cell_averages,axis=1)
df['sand'] = sand.apply(compute_cell_averages,axis=1)
df['po'] = po.apply(compute_cell_averages,axis=1)
df['pa'] = pa.apply(compute_cell_averages,axis=1)
df['ksat_0_5'] = ksat['0_5']
df['ksat_5_15'] = ksat['5_15']
df['ksat_15_30'] = ksat['15_30']
df['ksat_30_60'] = ksat['30_60']
df['ksat_60_100'] = ksat['60_100']
df['ksat_100_200'] = ksat['100_200']
df['m'] = ksat.apply(estimate_m,axis=1) # this doesn't look promising

In [9]:
# Read in soil type raster
rastpth = '/RHESSys/Como/auxdata/como_soils_30m_crop.tif'
ds = gdal.Open(rastpth)
soils = np.array(ds.GetRasterBand(1).ReadAsArray(),dtype='float')
soils[soils==-9999] = np.nan

In [10]:
m,n = soils.shape
df['soil'] = np.reshape(soils,(m*n,1))

In [11]:
# generate a list of soils types that fall only in the watershed
# Read in soil type raster
rastpth = '/RHESSys/Como/auxdata/como_soils_30m_crop_watershed.tif'
ds = gdal.Open(rastpth)
soils = np.array(ds.GetRasterBand(1).ReadAsArray(),dtype='float')
soils[soils==0] = np.nan

In [12]:
soils = np.unique(soils[np.isnan(soils)==0])

In [13]:
# generate a list of soils not contained within the watershed
all_soils = df.soil.unique() # grab all the soils in the data frame
idxs = []

for soil in all_soils: # loop through all the soils in the data frame
    tmp = []
    for s in soils: # loop through the soils in the watershed
        
        if s-soil == 0:
            tmp.append(1)
            
    if np.sum(tmp) > 0:
        idxs.append(0)
        
    else: idxs.append(1)
                
extra_soils = []

for idx,soil in zip(idxs,all_soils):
    if idx ==1: extra_soils.append(soil)

In [14]:
for soil in extra_soils:
    df.loc[df.soil==soil,:] = np.NaN

In [15]:
df.dropna(inplace=True)

In [17]:
out = df.groupby(by='soil').mean()

In [24]:
def estimate_m_2(df):
    ksats = np.array([df['ksat_0_5'],df['ksat_5_15'],df['ksat_15_30'],df['ksat_30_60'],df['ksat_60_100'],df['ksat_100_200']])
    surf = ksats[0]
    
    def exp_decay(z,m):
        return surf*np.exp((z/m)*-1)
    
    popt, pcov = curve_fit(exp_decay,mids,ksats)
    
    return popt[0]

In [25]:
out['m_2'] = out.apply(estimate_m_2,axis=1)

In [58]:
out.to_pickle('./data/como_soils.pcl')

In [79]:
def sum_texture(df):
    return df['silt']+df['sand']+df['clay']

In [91]:
out['texture_total'] = out.apply(sum_texture,axis=1)

In [75]:
out['silt'] = out.silt * (100./out.texture_total)
out['sand'] = out.sand * (100./out.texture_total)
out['clay'] = out.clay * (100./out.texture_total)

In [82]:
out['clay'] = out.clay.round()
out['silt'] = out.silt.round()
out['sand'] = out.sand.round()

In [86]:
def compute_sand(df):
    return 100. - np.sum([df['clay'],df['silt']])

In [90]:
out['sand'] = out.apply(compute_sand,axis=1)

In [102]:
cal = out.describe()[['ksat_0_5','po','pa']]

In [104]:
cal = cal.loc[['mean','min','max'],:]

In [106]:
cal['ksat_0_5'] /= 100.

In [111]:
cal.loc['max_scalar',:] = cal.loc['max']/cal.loc['mean']
cal.loc['min_scalar',:] = cal.loc['min']/cal.loc['mean']

In [112]:
cal

,ksat_0_5,po,pa
mean,0.307471,0.407403,13.761959
min,0.159092,0.398318,10.964427
max,0.383972,0.418298,16.293565
max_scalar,1.248808,1.026743,1.183957
min_scalar,0.517422,0.977701,0.796720


In [97]:
13.76/100.

0.1376

### Note:

It seems that given the above table, there isn't much variability in the soils across ComoCreek. I should probably just parameterize it as one soil....

In [93]:
df.describe()

,clay,silt,sand,po,pa,ksat_0_5,ksat_5_15,ksat_15_30,ksat_30_60,ksat_60_100,ksat_100_200,m,soil
count,16783.000000,16783.000000,16783.000000,16783.000000,16783.000000,16783.000000,16783.000000,16783.000000,16783.000000,16783.000000,16783.000000,1.678300e+04,16783.000000
mean,13.952544,20.716450,62.500349,0.407596,13.663587,30.403838,14.590718,7.683341,8.211930,9.855494,15.683045,1.178638e+08,140.396771
std,1.681907,1.140756,2.255130,0.011288,2.278666,8.903771,3.225571,1.067355,1.232863,2.410643,3.931845,1.094876e+08,12.939757
min,9.355786,16.207026,54.156929,0.368062,7.890588,9.261080,6.620969,4.788270,4.828485,4.051244,6.156418,1.033189e+02,123.000000
25%,12.576380,20.009050,61.080723,0.400451,11.785900,24.590642,12.212982,6.956430,7.366637,8.241879,12.854218,5.172685e+07,123.000000
50%,13.996609,20.809492,62.634343,0.409140,13.699515,31.854584,14.793041,7.669579,8.114238,9.578377,15.228470,1.028405e+08,145.000000
75%,15.140592,21.533541,64.141285,0.416134,15.323161,37.303087,17.063907,8.328271,8.946536,11.254957,17.958005,1.620404e+08,146.000000
max,19.684217,24.498658,69.403314,0.434209,20.496996,50.080135,23.790100,11.168921,13.176240,23.194210,38.690639,2.172282e+09,159.000000


In [48]:
def graph(df):
    ksats = df[['ksat_0_5','ksat_5_15','ksat_15_30','ksat_30_60','ksat_60_100','ksat_100_200']].as_matrix()
    m,n = ksats.shape
    
    ksats = np.reshape(ksats,(m*n))
    
    surf = ksats[0]
    fig = plt.figure(figsize=(5,5))
    ax = fig.add_subplot(111)
    
    ax.plot(ksats,mids,'.',label='Observed')
    
    depths = np.linspace(0,np.max(mids),100)
    
    def exp_decay(z,m):
        return surf*np.exp((z/m)*-1)
    
    ax.plot(exp_decay(depths,df['m_2'].as_matrix()[0]),depths,label='Modeled')
    

In [ ]:
tags = ['silt','sand','clay']